# Data visualization
In this notebook we have every function that plots something, let that be a histogram, a graph, anything. We are working with the following functions:
#### plot_graph(G)
    1. It takes a networkx graph object as an input and plots it
#### losses_epoch_plot
1) losses: list of the losses throughout the training session
2) save: boolean, decides if we should save the plot or not
3) save_name: The path where we save the plot if we want to save it
#### enhanced_histogram
1) data: the data we want to put on a histogram, typically the absolute value of the difference between the output and the test data
2) bins: the number of bins we want to divide the histogram into, default value is 30
3) save: boolean, decides if we should save the plot or not
4) save_name: The path where we save the plot if we want to save it
#### plot_graph_weight / plot_graph_weight_GEH / plot_graph_weight_forecast / plot_graph_weight_forecast_GEH
    1. These functions plots győr's map with colored edges based on the absolute/ GEH difference.
1) G: The graph we are ploting
2) model_output: The values calculated on the edges by out model
3) hidden_edges: Edges where we color our graph in case of forecasting we color the whole system
4) prepared_test_data_graph: The graph containing the correct egde weights
5) name: The plots name

In [1]:
#Necessary imports
import sumolib
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import random

import torch
import torch_geometric
import torch_geometric.data as Data
import torch_geometric.utils as pyg_utils

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import numpy as np
import json
import sys
import copy

In [4]:
def plot_graph(G):
    pos = nx.get_node_attributes(G, 'pos')
    nx.draw(G, pos, with_labels=False, node_size=1)
    plt.show()

In [5]:
def losses_epoch_plot(losses, save, save_name):
    fig = plt.figure()
    epochs = np.linspace(start=0, stop=np.size(losses), num=np.size(losses))
    plt.scatter(epochs,losses,c='red')
    plt.ylabel('Loss', fontsize=20)
    plt.xlabel('Epochs', fontsize=20)
    plt.grid(True,which='major',color='gray')
    plt.minorticks_on()
    plt.tick_params(axis='both', which='both', direction= 'in')
    plt.ylim([0,0.2])
    fig.set_size_inches(14,7.5)
    if save:
        plt.savefig(save_name)

In [6]:
def enhanced_histogram(data, bins=30, save=False, save_name='histogram.png'):
    fig = plt.figure()
    plt.hist(data, bins=bins, color='blue', edgecolor='black')
    plt.ylabel('Frequency', fontsize=20)
    plt.xlabel('Difference', fontsize=20)  # Changed to 'Difference' to reflect the data
    plt.grid(True, which='major', color='gray', linestyle='--', linewidth=0.7)
    plt.minorticks_on()
    plt.grid(True, which='minor', color='lightgray', linestyle=':', linewidth=0.5)
    plt.tick_params(axis='both', which='both', direction='in', top=True, right=True)
    fig.set_size_inches(14, 7.5)
    plt.title('Histogram of Differences', fontsize=22)  # Changed title to reflect the data

    if save:
        plt.savefig(save_name)
    plt.show()

In [1]:
def plot_graph_weight(G, model_output, prepared_test_data_graph, name):
    matplotlib.rcParams.update({'font.size': 28})

    plt.figure(figsize=(16,9))

    pos = nx.get_node_attributes(G, 'pos')
    N = len(hidden_edges)
    #print(N)
    edges_to_hide_sort = hidden_edges
    edges_to_hide_sort.sort()
    difference = np.zeros(N)
    #print(edges_to_hide_sort)
    for i in range(N):
        if float(model_output[edges_to_hide_sort][i])<0:
            model_output[edges_to_hide_sort][i] = 0
        difference[i] = float(abs(model_output[edges_to_hide_sort][i] - prepared_test_data_graph.edge_attr[edges_to_hide_sort][i]))*(2*1800)/4
        #print(model_output[edges_to_hide_sort][i])

    maxdiff = max(difference)
    mindiff = 0
    maxdiff_int=int(maxdiff)
    #print(maxdiff)
    average = sum(difference) / len(difference)
    
    c = np.arange(10, maxdiff_int + 1)


    cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["green", "yellow", "red"])
     
    colors = [cmap(c/maxdiff) for c in difference]
    

    edge_colors = {edge: 'black' for edge in G.edges()}


    edges_to_color_edges = []
    edges=list(G.edges())
    edges_to_color_by_id=[]

    for i in edges_to_hide_sort:
        edges_to_color_edges.append(edges[i])
    for i in edges_to_color_edges:
        edges_to_color_by_id.append(G.edges[i]['edge_id'])
    #rint(edges_to_color_by_id)
    #print(len(edges_to_color_by_id))
    colorindexiter=0
    for edge in G.edges():
        edge_id = G.edges[edge]['edge_id']



        if edge_id in edges_to_color_by_id:

            edge_colors[edge] = colors[colorindexiter]
            colorindexiter+=1

    plt.axis([250, 3500, -500, 2500])
    nx.set_edge_attributes(G, edge_colors, 'color')

    edge_color_values = [G[u][v]['color'] for u, v in G.edges()]
    options = {
        

        "width": 3,
        "edge_cmap": plt.cm.Blues,
        "with_labels": False,
    }
    nx.draw(G, pos, **options, edge_color=edge_color_values, node_size=0) 
    
    norm = matplotlib.colors.Normalize(vmin=mindiff, vmax=maxdiff) 
  
    # creating ScalarMappable 
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm) 
    sm.set_array([]) 
    
    plt.colorbar(sm, ax=plt.gca(),ticks=np.linspace(mindiff, maxdiff,10)) 
    plt.title('Differences depicted on the graph with' + str(name) + "\n" + "Average:" + str(average))
    plt.show()
    return maxdiff

In [16]:
def GEH(M, C):
    geh = np.sqrt(2*np.power((float(M) - float(C)), 2) / (float(M) + float(C)))
    return geh

In [51]:
def plot_graph_weight_GEH(G,model_output,hidden_edges, prepared_test_data_graph, name):
    matplotlib.rcParams.update({'font.size': 28})

    plt.figure(figsize=(16,9))
    """ fig, ax = plt.subplots(dpi=100)
    mycax=ax.inset_axes[0.95, 0.1, 0.05, 0.8] """ 
    """ cax = plt.axes((0.85, 0.1, 0.075, 0.8)) """
    
    pos = nx.get_node_attributes(G, 'pos')

    N = len(hidden_edges)
    #print(N)
    edges_to_hide_sort = hidden_edges
    edges_to_hide_sort.sort()
    difference = np.zeros(N)
    #print(edges_to_hide_sort)

    for i in range(N):
        if model_output[edges_to_hide_sort][i]<0:
            model_output[edges_to_hide_sort][i] = 0
        M = float(model_output[edges_to_hide_sort][i]*(2*1800)/4)

        C = float(prepared_test_data_graph.edge_attr[edges_to_hide_sort][i]*(2*1800)/4)

        difference[i] = GEH(M, C)
 
    #print(difference)
    maxdiff = max(difference)
    mindiff = 0
    maxdiff_int=int(maxdiff)
    cleanedList= [x for x in difference if ~np.isnan(x)]
    average = sum(cleanedList) / len(cleanedList)
    c = np.arange(10, maxdiff_int + 1)


    cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["green", "yellow", "red"])
     
    colors = [cmap(c/maxdiff) for c in difference]
    

    edge_colors = {edge: 'black' for edge in G.edges()}


    edges_to_color_edges = []
    edges=list(G.edges())
    edges_to_color_by_id=[]

    for i in edges_to_hide_sort:
        edges_to_color_edges.append(edges[i])
    for i in edges_to_color_edges:
        edges_to_color_by_id.append(G.edges[i]['edge_id'])

    colorindexiter=0
    for edge in G.edges():
        edge_id = G.edges[edge]['edge_id']


        if edge_id in edges_to_color_by_id:

            edge_colors[edge] = colors[colorindexiter]
            colorindexiter+=1

    plt.axis([250, 3500, -500, 2500])
    nx.set_edge_attributes(G, edge_colors, 'color')

    edge_color_values = [G[u][v]['color'] for u, v in G.edges()]
    options = {
        

        "width": 3,
        "edge_cmap": plt.cm.Blues,
        "with_labels": False,
    }
    nx.draw(G, pos, **options, edge_color=edge_color_values, node_size=0) 
    
    norm = matplotlib.colors.Normalize(vmin=mindiff, vmax=maxdiff) 
  
    # creating ScalarMappable 
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm) 
    sm.set_array([]) 
    
    plt.colorbar(sm, ax=plt.gca(),ticks=np.linspace(mindiff, maxdiff,10)) 
    plt.title('GEH Differences depicted on the graph with' + str(name) + "\n" + "Average:" + str(average))
    plt.show()
    return maxdiff

In [ ]:
def plot_graph_weight_forecast(G, model_output, hidden_edges, prepared_test_data_graph, name):
    matplotlib.rcParams.update({'font.size': 28})

    plt.figure(figsize=(16,9))

    pos = nx.get_node_attributes(G, 'pos')
    N = 503
    #print(N)
    edges_to_hide_sort = []
    for i in range(N):
        edges_to_hide_sort.append(i)
    
    difference = np.zeros(N)
    #print(edges_to_hide_sort)
    for i in range(N):
        if float(model_output[edges_to_hide_sort][i])<0:
            model_output[edges_to_hide_sort][i] = 0
        difference[i] = float(abs(model_output[edges_to_hide_sort][i] - prepared_test_data_graph.edge_attr[edges_to_hide_sort][i]))*(2*1800)/4
        #print(model_output[edges_to_hide_sort][i])

    maxdiff = max(difference)
    mindiff = 0
    maxdiff_int=int(maxdiff)
    #print(maxdiff)
    average = sum(difference) / len(difference)
    
    c = np.arange(10, maxdiff_int + 1)


    cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["green", "yellow", "red"])
     
    colors = [cmap(c/maxdiff) for c in difference]
    

    edge_colors = {edge: 'black' for edge in G.edges()}


    edges_to_color_edges = []
    edges=list(G.edges())
    edges_to_color_by_id=[]

    for i in edges_to_hide_sort:
        edges_to_color_edges.append(edges[i])
    for i in edges_to_color_edges:
        edges_to_color_by_id.append(G.edges[i]['edge_id'])
    #rint(edges_to_color_by_id)
    #print(len(edges_to_color_by_id))
    colorindexiter=0
    for edge in G.edges():
        edge_id = G.edges[edge]['edge_id']



        if edge_id in edges_to_color_by_id:

            edge_colors[edge] = colors[colorindexiter]
            colorindexiter+=1

    plt.axis([250, 3500, -500, 2500])
    nx.set_edge_attributes(G, edge_colors, 'color')

    edge_color_values = [G[u][v]['color'] for u, v in G.edges()]
    options = {
        

        "width": 3,
        "edge_cmap": plt.cm.Blues,
        "with_labels": False,
    }
    nx.draw(G, pos, **options, edge_color=edge_color_values, node_size=0) 
    
    norm = matplotlib.colors.Normalize(vmin=mindiff, vmax=maxdiff) 
  
    # creating ScalarMappable 
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm) 
    sm.set_array([]) 
    
    plt.colorbar(sm, ax=plt.gca(),ticks=np.linspace(mindiff, maxdiff,10)) 
    plt.title('Differences depicted on the graph with' + str(name) + "\n" + "Average:" + str(average))
    plt.show()
    return maxdiff

In [ ]:
def plot_graph_weight_forecast_GEH(G, model_output, hidden_edges, prepared_test_data_graph, name):
    matplotlib.rcParams.update({'font.size': 28})

    plt.figure(figsize=(16,9))

    pos = nx.get_node_attributes(G, 'pos')
    N = len(list(G.edges()))
    #print(N)
    edges_to_hide_sort = []
    for i in range(N):
        edges_to_hide_sort.append(i)
    
    difference = np.zeros(N)
    #print(edges_to_hide_sort)
    for i in range(N):
        if model_output[edges_to_hide_sort][i]<0:
            model_output[edges_to_hide_sort][i] = 0
        M = float(model_output[edges_to_hide_sort][i]*(2*1800)/4)

        C = float(prepared_test_data_graph.edge_attr[edges_to_hide_sort][i]*(2*1800)/4)

        difference[i] = GEH(M, C)

    maxdiff = max(difference)
    mindiff = 0
    maxdiff_int=int(maxdiff)
    #print(maxdiff)
    cleanedList= [x for x in difference if ~np.isnan(x)]
    average = sum(cleanedList) / len(cleanedList)
    c = np.arange(10, maxdiff_int + 1)


    cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["green", "yellow", "red"])
     
    colors = [cmap(c/maxdiff) for c in difference]
    

    edge_colors = {edge: 'black' for edge in G.edges()}


    edges_to_color_edges = []
    edges=list(G.edges())
    edges_to_color_by_id=[]

    for i in edges_to_hide_sort:
        edges_to_color_edges.append(edges[i])
    for i in edges_to_color_edges:
        edges_to_color_by_id.append(G.edges[i]['edge_id'])
    #rint(edges_to_color_by_id)
    #print(len(edges_to_color_by_id))
    colorindexiter=0
    for edge in G.edges():
        edge_id = G.edges[edge]['edge_id']



        if edge_id in edges_to_color_by_id:

            edge_colors[edge] = colors[colorindexiter]
            colorindexiter+=1

    plt.axis([250, 3500, -500, 2500])
    nx.set_edge_attributes(G, edge_colors, 'color')

    edge_color_values = [G[u][v]['color'] for u, v in G.edges()]
    options = {
        

        "width": 3,
        "edge_cmap": plt.cm.Blues,
        "with_labels": False,
    }
    nx.draw(G, pos, **options, edge_color=edge_color_values, node_size=0) 
    
    norm = matplotlib.colors.Normalize(vmin=mindiff, vmax=maxdiff) 
  
    # creating ScalarMappable 
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm) 
    sm.set_array([]) 
    
    plt.colorbar(sm, ax=plt.gca(),ticks=np.linspace(mindiff, maxdiff,10)) 
    plt.title('Differences depicted on the graph with' + str(name) + "\n" + "Average:" + str(average))
    plt.show()
    return maxdiff